In [109]:
import pandas as pd

In [110]:
#open a the file from bpost and lowercase all  
bpost_codes = pd.read_csv("data/raw/zipcodes_alpha_nl_new.csv", delimiter=";")
bpost_codes[['Plaatsnaam', 'Hoofdgemeente', 'Provincie']] = bpost_codes[['Plaatsnaam', 'Hoofdgemeente', 'Provincie']].apply(lambda x: x.astype(str).str.lower())


In [117]:
#open a the scraped files and lowercase all 
def open_and_lowercase(filename:str):
    df = pd.read_csv(filename) 
    df["locality_name"] = df["locality_name"].str.lower()
    return df

apt_file = "data/raw/raw_apartement_te_koop.csv"
house_file = "data/raw/raw_huis_te_koop.csv"
postal_codes_immoweb_apt = open_and_lowercase(apt_file)
postal_codes_immoweb_house = open_and_lowercase(house_file)



In [119]:
# drop non belgian properties
def drop_non_belgian(df):
    df['is_belgian'] = df['postal_code'].astype(str).isin(bpost_codes['Postcode'].astype(str))

    df.loc[df['is_belgian']==False]

    df = df.drop(df[df['is_belgian'] == False].index)
    return df

postal_codes_immoweb_apt = drop_non_belgian(postal_codes_immoweb_apt)
postal_codes_immoweb_house = drop_non_belgian(postal_codes_immoweb_house)


In [ ]:
#translate all city names to dutch
georef = pd.read_csv("data/raw/georef-belgium-postal-codes.csv", delimiter=";")
georef[['Sub-municipality name (French)','Sub-municipality name (Dutch)', 'Sub-municipality name (German)']] = georef[['Sub-municipality name (French)','Sub-municipality name (Dutch)', 'Sub-municipality name (German)']].apply(lambda x: x.str.lower())
#if locality_name = Sub-municipality name (French) of Sub-municipality name (German), replace it with Sub-municipality name (Dutch) if it's not NaN
georef_with_nl = georef[~georef['Sub-municipality name (Dutch)'].isna()]
locality_name_was_french = (postal_codes_immoweb_house["locality_name"].isin(georef_with_nl['Sub-municipality name (French)']))
locality_name_was_german = (postal_codes_immoweb_house["locality_name"].isin(georef_with_nl['Sub-municipality name (German)']))
df_fr_de = postal_codes_immoweb_house[locality_name_was_french | locality_name_was_german]

def get_dutch_locality_name(current_locality_name):
    name_loc = georef_with_nl.loc[(georef_with_nl["Sub-municipality name (French)"] == current_locality_name) | (georef_with_nl["Sub-municipality name (German)"] == current_locality_name)]
    if not name_loc.empty:
        return(name_loc["Sub-municipality name (Dutch)"].iloc[0])
    else:
        return current_locality_name
    
postal_codes_immoweb_house["locality_name"] = postal_codes_immoweb_house["locality_name"].apply(get_dutch_locality_name)


postal_codes_immoweb_apt["locality_name"] =  postal_codes_immoweb_apt["locality_name"].apply(get_dutch_locality_name)


In [133]:
#save to csv

def save_df_to_csv(df, filepath):
    df.to_csv(filepath)

file_name_apt = "data/raw/postal_code_dutch_location_apartement_te_koop.csv"
file_name_house = "data/raw/postal_code_dutch_location_huis_te_koop.csv"

save_df_to_csv(postal_codes_immoweb_house, file_name_house)
save_df_to_csv(postal_codes_immoweb_apt, file_name_apt)